## Prophet Model

## Imports

In [ ]:
# Import the required libraries and dependencies
import pandas as pd
import hvplot.pandas
import datetime as dt
import holoviews as hv
from prophet import Prophet
from pathlib import Path
from plotting import Plotter
%matplotlib inline

## Data

In [ ]:
etf_data = pd.read_csv(Path("Resources/Data/etf_data.csv"), index_col="Date", infer_datetime_format=True, parse_dates=True)
etf_data.head()

In [ ]:
# Setting global variables
tickers = ['ARKK','SPY','FNGU']
# Separating ETF DataFrame into separate DataFrames
df_ARKK,df_SPY,df_FNGU = [etf_data[i].to_frame(i) for i in tickers]

In [ ]:
hv.extension("bokeh")

df_ARKK['ARKK'].hvplot()

## Create Model

In [ ]:
# Reset the index so that we recode the columns for Prophet
df_ARKK=df_ARKK.reset_index()
df_SPY=df_SPY.reset_index()
df_FNGU=df_FNGU.reset_index()

In [ ]:
# Label the columns ds and y so that the syntax is recognized by Prophet
df_ARKK.columns=['ds','y']
df_SPY.columns=['ds', 'y']
df_FNGU.columns=['ds', 'y']

In [ ]:
# View dataframe shape, and the first and last five rows of the mercado_prophet_df DataFrame
display(df_ARKK.tail())
display(df_SPY.head())
display(df_FNGU.head())

## Train Model

In [ ]:
# Sort the DataFrame by `ds` in ascending order
df_ARKK=df_ARKK.sort_values(by=['ds'], ascending=True)
df_SPY=df_SPY.sort_values(by=['ds'], ascending=True)
df_FNGU=df_FNGU.sort_values(by=['ds'], ascending=True)

In [ ]:
# Set `bokeh` to render hvPlot charts
hv.extension("bokeh")

# Diplay the data using hvPlot
df_ARKK.hvplot()

In [ ]:
# Call the Prophet function, store as an object
model_ARKK=Prophet()
model_SPY=Prophet()
model_FNGU=Prophet()

In [ ]:
# Fit the Prophet model. Cut last 60 days of data
model_ARKK.fit(df_ARKK[:-60])
model_SPY.fit(df_SPY[:-60])
model_FNGU.fit(df_FNGU[:-60])

In [ ]:
plotter = Plotter('Prophet')

## Forecast Model

In [ ]:
# Create a future dataframe to hold predictions
# Make the prediction go out as far as 60 days
future_trends_ARKK = model_ARKK.make_future_dataframe(periods=86, freq='D')
future_trends_SPY = model_SPY.make_future_dataframe(periods=86, freq='D')
future_trends_FNGU = model_FNGU.make_future_dataframe(periods=86, freq='D')

In [ ]:
# Make the predictions for the trend data using the future_trends DataFrame
forecast_trends_ARKK = model_ARKK.predict(future_trends_ARKK)
forecast_trends_SPY = model_SPY.predict(future_trends_SPY)
forecast_trends_FNGU = model_FNGU.predict(future_trends_FNGU)

In [ ]:
def filter_days(df):

    df = df[df['ds'].dt.dayofweek < 5]

    df.drop(
        labels=[1005, 1040],
        axis=0,
        inplace=True
    )
    return df


In [ ]:
forecast_trends_ARKK,forecast_trends_SPY,forecast_trends_FNGU=[filter_days(i) for i in [forecast_trends_ARKK,forecast_trends_SPY,forecast_trends_FNGU]]

In [ ]:
forecast_trends_FNGU

In [ ]:
# Plot the Prophet predictions for the ARKK trends data
#model_ARKK.plot(forecast_trends_ARKK)
# model_SPY.plot(forecast_trends_SPY)
# model_FNGU.plot(forecast_trends_FNGU)


In [ ]:
# Use the plot_components function to visualize the forecast results 
#figures_ARKK = model_ARKK.plot_components(forecast_trends_ARKK)

In [ ]:
# At this point, it's useful to set the `datetime` index of the forecast data.
forecast_trends_ARKK = forecast_trends_ARKK.set_index(['ds'])
forecast_trends_SPY = forecast_trends_SPY.set_index(['ds'])
forecast_trends_FNGU = forecast_trends_FNGU.set_index(['ds'])

## Evaluation

### ARKK

In [ ]:
# From the `forecast_trends` DataFrame, use hvPlot to visualize
#  the yhat, yhat_lower, and yhat_upper columns over the last 60 days 
hv.extension("bokeh")
forecast_trends_ARKK[['yhat', 'yhat_lower', 'yhat_upper']].iloc[-60:].hvplot()

In [ ]:
# Create a `forecast_60_days_ARKK` Dataframe, which contains just forecasts for that month
# The DataFrame should include the columns yhat_upper, yhat_lower, and yhat
forecast_60_days_ARKK = forecast_trends_ARKK.loc['2022-01-04':'2022-03-30'][['yhat_upper', 'yhat_lower', 'yhat']]
# Replace the column names to something less technical sounding
forecast_60_days_ARKK = forecast_60_days_ARKK.rename(
    columns={
        'yhat_upper':'Best Case',
        'yhat_lower':'Worst Case',
        'yhat':'Most Likely Case'
    }
)

### SPY

In [ ]:
# From the `forecast_trends` DataFrame, use hvPlot to visualize
#  the yhat, yhat_lower, and yhat_upper columns over the last 60 days 
hv.extension("bokeh")
forecast_trends_SPY[['yhat', 'yhat_lower', 'yhat_upper']].iloc[-60:].hvplot()

In [ ]:
# Create a `forecast_60_days_ARKK` Dataframe, which contains just forecasts for that month
# The DataFrame should include the columns yhat_upper, yhat_lower, and yhat
forecast_60_days_SPY = forecast_trends_SPY.loc['2022-01-04':'2022-03-30'][['yhat_upper', 'yhat_lower', 'yhat']]
# Replace the column names to something less technical sounding
forecast_60_days_SPY = forecast_60_days_SPY.rename(
    columns={
        'yhat_upper':'Best Case',
        'yhat_lower':'Worst Case',
        'yhat':'Most Likely Case'
    }
)

### FNGU

In [ ]:
# From the `forecast_trends` DataFrame, use hvPlot to visualize
#  the yhat, yhat_lower, and yhat_upper columns over the last 60 days 
hv.extension("bokeh")
forecast_trends_FNGU[['yhat', 'yhat_lower', 'yhat_upper']].iloc[-60:].hvplot()

In [ ]:
# Create a `forecast_60_days_ARKK` Dataframe, which contains just forecasts for that month
# The DataFrame should include the columns yhat_upper, yhat_lower, and yhat
forecast_60_days_FNGU = forecast_trends_FNGU.loc['2022-01-04':'2022-03-30'][['yhat_upper', 'yhat_lower', 'yhat']]
# Replace the column names to something less technical sounding
forecast_60_days_FNGU = forecast_60_days_FNGU.rename(
    columns={
        'yhat_upper':'Best Case',
        'yhat_lower':'Worst Case',
        'yhat':'Most Likely Case'
    }
)


### Mean Absolute Error

In [ ]:
forecast_60_days_ARKK.mean()

In [ ]:
forecast_60_days_SPY.mean()

In [ ]:
forecast_60_days_FNGU.mean()

### Mean Squared Error

### Root Mean Squared Error

## Results

### Actual vs Forecast

In [ ]:
# Helper method to display DataFrame head and tail
display_head_tail = lambda df: display(df.head(),df.tail())
# Variable for titles
avf = 'Actual vs Forecasted'
# Helper method to get titles
get_avf_title = lambda index: tickers[index] + ' ' + avf


In [ ]:
def get_actual_forecasted(df, ticker, forecast_60_days):
    

    df.index=df['ds']
    df_actual=df[-60:]


    actual_forecasted = pd.concat([forecast_60_days, df_actual], axis=1, join="inner")

    actual_forecasted.drop(columns=['Best Case', 'Most Likely Case', 'ds'], inplace=True)
    actual_forecasted.rename(columns={'Worst Case':f'{ticker} Forecasted','y':f'{ticker} Actual'}, inplace=True)
    actual_forecasted[f'{ticker} Forecasted']= actual_forecasted[f'{ticker} Forecasted'].apply(lambda x: round(x,2))
    
    return actual_forecasted


In [ ]:
ARKK_actual_forecasted=get_actual_forecasted(df_ARKK, 'ARKK', forecast_60_days_ARKK)

In [ ]:
# Plotting ARKK actual vs forecasted adjusted closing prices
plotter.line(ARKK_actual_forecasted, get_avf_title(0)) 

In [ ]:
SPY_actual_forecasted=get_actual_forecasted(df_SPY, 'SPY', forecast_60_days_SPY)

In [ ]:
# Plotting ARKK actual vs forecasted adjusted closing prices
plotter.line(SPY_actual_forecasted, get_avf_title(1)) 

In [ ]:
FNGU_actual_forecasted=get_actual_forecasted(df_FNGU, 'FNGU', forecast_60_days_FNGU)

In [ ]:
# Plotting ARKK actual vs forecasted adjusted closing prices
plotter.line(FNGU_actual_forecasted, get_avf_title(2)) 

### Training Data vs Actual vs Forecast

## Write-Up

## Conclusion